Download the Video Game Sales with Ratings dataset from this link: Video Game Sales with Ratings.
https://www.kaggle.com/datasets/rush4ratio/video-game-sales-with-ratings

* Load the dataset as a Pandas data frame.
* Display the first ten rows of data.
* Find the dimensions (number of rows and columns) in the data frame. What do these two numbers represent in the context of the data?
* Find the top five games by critic score.
* Find the number of video games in the data frame in each genre.
* Find the first five games in the data frame on the SNES platform.
* Find the five publishers with the highest total global sales. Note: You will need to calculate the total global sales for each publisher to do this.
* Create a new column in the data frame that calculates the percentage of global sales from North America. Display the first five rows of the new data frame.
* Find the number NaN entries (missing data values) in each column.
* Try to calculate the median user score of all the video games. You will likely run into an error because some of the user score entries are a non-numerical string that cannot be converted to a float. Find and replace this string with NaN and then calculate the median. Then, replace all NaN entries in the user score column with the median value.

In [1]:
import time

# file system searches etc
import os
from os.path import basename, exists

# data frames
import pandas as pd

# web access and html parsing (urllib, its submodules)
import requests
import urllib
import urllib.request
import urllib.error
from urllib.request import urlretrieve
# for a workaround.
import ssl

# work around read_html deprecation issue
from io import StringIO

# fancy table printing
from tabulate import tabulate

In [2]:
# For testing: Make warnings fatal.

import warnings
warnings.filterwarnings("error")

In [8]:
# Store final project data in its own directory.

DATA_DIRECTORY = "data"

# Check if the directory exists
if not os.path.exists(DATA_DIRECTORY):
    # If it doesn't exist, make it
    os.makedirs(DATA_DIRECTORY)
    print(f"Directory '{DATA_DIRECTORY}' created.")



Directory 'data' created.


In [9]:
# constants for accessing file.

VIDEO_GAME_CSV_FILE = DATA_DIRECTORY+"/video_game_rankings.csv"
VIDEO_GAME_URL = "https://drive.google.com/uc?id=1_8go9zumRg0gMaLnroUc853I4cI1rVHQ&export=download"


In [10]:
# web download function

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36"
}

# download: A good citizen download function
#     url - the url accessed
#     destination - local file to write
#
# respects code 429 and waits instead of pounding.

# Function to disable SSL certificate verification
def disable_ssl_verification():
    ssl._create_default_https_context = ssl._create_unverified_context

# Call the function to disable SSL verification
# This is to workaround an SSL certificate error I am getting.
disable_ssl_verification()

def download(url, destination, secure=True):
    try:
        # Send a GET request with headers
        response = requests.get(url, headers=headers, verify=secure)
        # Check if the request was successful
        if response.status_code == 200:
            with open(destination, 'w') as f:
                f.write(response.text)
            print("Downloaded " + destination)
        elif response.status_code == 429:
            # Extract the Retry-After header value
            # This is to avoid hammering sites.
            retry_after = response.headers.get("Retry-After")
            if retry_after:
                # Convert the Retry-After value to seconds
                retry_after_seconds = int(retry_after)
                print("Rate limit exceeded. Waiting for " + str(retry_after_seconds) + " seconds.")
                time.sleep(retry_after_seconds)
                # Retry the request after waiting
                download(url, destination)
            else:
                print("Rate limit exceeded. Retry-After header not found.")
        else:
            print("Website returned " + str(response.status_code))
    except urllib.error.HTTPError:
        print("Failed to download " + url)
    except Exception:
        print("Error writing " + destination)
    return

def downloadFile(url, filename):
    if not exists(filename):
        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local + "\n")
        return local, _

def downloadRawFile(url, filename):
    if not os.path.exists(filename):
        # Modify the URL to the raw content URL (replace "github.com" with "raw.githubusercontent.com")
        raw_url = url + "?raw=true"
        # Download the raw content
        local, _ = downloadFile(raw_url, filename)


* Load the dataset as a Pandas data frame.


In [11]:
downloadRawFile(VIDEO_GAME_URL, VIDEO_GAME_CSV_FILE)    


Downloaded data/video_game_rankings.csv



* Display the first ten rows of data.


In [21]:
def pretty_print_df(df, rows=None, interesting_columns=None):
    if rows is not None:
        df = df.head(rows)  # rows parameter limits the DataFrame to the specified number of rows
    if interesting_columns:
        # Filter DataFrame to include only specified columns
        df = df[interesting_columns]    
    # Use Tabulate pretty printing.
    print(tabulate(df, headers='keys', tablefmt='pretty', showindex=False))
    
# load csv into pandas data frame
video_game_rankings_df = pd.read_csv(VIDEO_GAME_CSV_FILE)

# show 10 rows
pretty_print_df(video_game_rankings_df, rows=10)


+---------------------------+----------+-----------------+--------------+-----------+----------+----------+----------+-------------+--------------+--------------+--------------+------------+------------+-----------+--------+
|           Name            | Platform | Year_of_Release |    Genre     | Publisher | NA_Sales | EU_Sales | JP_Sales | Other_Sales | Global_Sales | Critic_Score | Critic_Count | User_Score | User_Count | Developer | Rating |
+---------------------------+----------+-----------------+--------------+-----------+----------+----------+----------+-------------+--------------+--------------+--------------+------------+------------+-----------+--------+
|        Wii Sports         |   Wii    |     2006.0      |    Sports    | Nintendo  |  41.36   |  28.96   |   3.77   |    8.45     |    82.53     |     76.0     |     51.0     |     8      |   322.0    | Nintendo  |   E    |
|     Super Mario Bros.     |   NES    |     1985.0      |   Platform   | Nintendo  |  29.08   |   3

* Find the dimensions (number of rows and columns) in the data frame. What do these two numbers represent in the context of the data?


In [25]:
def pretty_print_df_info(df):
    # Show the column names and data types.
    columns_info = df.dtypes.reset_index()
    columns_info.columns = ['Column', 'Data Type']
    table = tabulate(columns_info, headers='keys', tablefmt='pretty', showindex=False)
    print(f"Dataframe Information:\n{table}")
    

print("\n")
pretty_print_df_info(video_game_rankings_df)
print("\n")
print(f"There are {video_game_rankings_df.shape[0]} video game rows and {video_game_rankings_df.shape[1]} columns of information.")




Dataframe Information:
+-----------------+-----------+
|     Column      | Data Type |
+-----------------+-----------+
|      Name       |  object   |
|    Platform     |  object   |
| Year_of_Release |  float64  |
|      Genre      |  object   |
|    Publisher    |  object   |
|    NA_Sales     |  float64  |
|    EU_Sales     |  float64  |
|    JP_Sales     |  float64  |
|   Other_Sales   |  float64  |
|  Global_Sales   |  float64  |
|  Critic_Score   |  float64  |
|  Critic_Count   |  float64  |
|   User_Score    |  object   |
|   User_Count    |  float64  |
|    Developer    |  object   |
|     Rating      |  object   |
+-----------------+-----------+


There are 16719 video game rows and 16 columns of information.


* Find the top five games by critic score.


In [32]:
# Fix the Year_Of_Release field data type:

video_game_rankings_df['Year_of_Release'] = video_game_rankings_df['Year_of_Release'].astype('Int64')

sorted_by_critic_score_df = video_game_rankings_df.sort_values(by='Critic_Score', ascending=False)

interesting_fields = ['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher', 'Critic_Score']

# show 5 rows
pretty_print_df(sorted_by_critic_score_df, rows=5, interesting_columns=interesting_fields)

+--------------------------+----------+-----------------+----------+----------------------+--------------+
|           Name           | Platform | Year_of_Release |  Genre   |      Publisher       | Critic_Score |
+--------------------------+----------+-----------------+----------+----------------------+--------------+
| Tony Hawk's Pro Skater 2 |    PS    |      2000       |  Sports  |      Activision      |     98.0     |
|   Grand Theft Auto IV    |   PS3    |      2008       |  Action  | Take-Two Interactive |     98.0     |
|   Grand Theft Auto IV    |   X360   |      2008       |  Action  | Take-Two Interactive |     98.0     |
|       SoulCalibur        |    DC    |      1999       | Fighting |  Namco Bandai Games  |     98.0     |
|    Grand Theft Auto V    |   XOne   |      2014       |  Action  | Take-Two Interactive |     97.0     |
+--------------------------+----------+-----------------+----------+----------------------+--------------+


* Find the number of video games in the data frame in each genre.


In [33]:
def sum_column_and_pretty_print_df(df, column_name):
    # Get a sum of the count for each unique value in the column_name parameter.
    summary = df[column_name].value_counts().reset_index()
    summary.columns = [column_name, 'Count']

    # Pretty print result.
    print(tabulate(summary, headers='keys', tablefmt='pretty', showindex=False))

sum_column_and_pretty_print_df(video_game_rankings_df, 'Genre')

+--------------+-------+
|    Genre     | Count |
+--------------+-------+
|    Action    | 3370  |
|    Sports    | 2348  |
|     Misc     | 1750  |
| Role-Playing | 1500  |
|   Shooter    | 1323  |
|  Adventure   | 1303  |
|    Racing    | 1249  |
|   Platform   |  888  |
|  Simulation  |  874  |
|   Fighting   |  849  |
|   Strategy   |  683  |
|    Puzzle    |  580  |
+--------------+-------+


* Find the first five games in the data frame on the SNES platform.


In [34]:
def filter_by_column_value_and_pretty_print_df(df, column_name, value, rows, interesting_columns=None):
    # Do dataframe filtering based on the column name and the value.
    filtered_df = df[df[column_name] == value].head(rows)
    # Pretty print result.
    pretty_print_df(filtered_df, rows=rows, interesting_columns=interesting_columns)

filter_by_column_value_and_pretty_print_df(video_game_rankings_df, 'Platform', 'SNES', rows=5, interesting_columns = interesting_fields)

+--------------------------------------+----------+-----------------+----------+-----------+--------------+
|                 Name                 | Platform | Year_of_Release |  Genre   | Publisher | Critic_Score |
+--------------------------------------+----------+-----------------+----------+-----------+--------------+
|          Super Mario World           |   SNES   |      1990       | Platform | Nintendo  |     nan      |
|        Super Mario All-Stars         |   SNES   |      1993       | Platform | Nintendo  |     nan      |
|         Donkey Kong Country          |   SNES   |      1994       | Platform | Nintendo  |     nan      |
|           Super Mario Kart           |   SNES   |      1992       |  Racing  | Nintendo  |     nan      |
| Street Fighter II: The World Warrior |   SNES   |      1992       | Fighting |  Capcom   |     nan      |
+--------------------------------------+----------+-----------------+----------+-----------+--------------+


* Find the five publishers with the highest total global sales. 
- Note: You will need to calculate the total global sales for each publisher to do this.


In [36]:
def sum_sales_column_and_pretty_print_df(df, summary_column_name, sales_column_name):
    # Use groupby for the specified summary column and calculate the sum of sales_column_name
    summary_df = df.groupby(summary_column_name)[sales_column_name].sum().reset_index()
    # Sort the DataFrame by total global sales in descending order
    summary_df = summary_df.sort_values(by=sales_column_name, ascending=False)
    # Return the summary DataFrame if needed
    return summary_df

summary_df = sum_sales_column_and_pretty_print_df(video_game_rankings_df, 'Publisher', 'Global_Sales')
# Pretty print the result
print(tabulate(summary_df.head(5), headers='keys', tablefmt='pretty', showindex=False))


+-----------------------------+--------------+
|          Publisher          | Global_Sales |
+-----------------------------+--------------+
|          Nintendo           |   1788.81    |
|       Electronic Arts       |   1116.96    |
|         Activision          |    731.16    |
| Sony Computer Entertainment |    606.48    |
|           Ubisoft           |    471.61    |
+-----------------------------+--------------+


* Create a new column in the data frame that calculates the percentage of global sales from North America. 
* Display the first five rows of the new data frame.


In [37]:
video_game_rankings_df['NA_Sales_Percentage'] = ((video_game_rankings_df['NA_Sales'] / video_game_rankings_df['Global_Sales']) * 100).round(2)
interesting_fields = ['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher', 'Global_Sales', 'NA_Sales', 'NA_Sales_Percentage']
pretty_print_df(video_game_rankings_df, rows=5, interesting_columns=interesting_fields)

+--------------------------+----------+-----------------+--------------+-----------+--------------+----------+---------------------+
|           Name           | Platform | Year_of_Release |    Genre     | Publisher | Global_Sales | NA_Sales | NA_Sales_Percentage |
+--------------------------+----------+-----------------+--------------+-----------+--------------+----------+---------------------+
|        Wii Sports        |   Wii    |      2006       |    Sports    | Nintendo  |    82.53     |  41.36   |        50.12        |
|    Super Mario Bros.     |   NES    |      1985       |   Platform   | Nintendo  |    40.24     |  29.08   |        72.27        |
|      Mario Kart Wii      |   Wii    |      2008       |    Racing    | Nintendo  |    35.52     |  15.68   |        44.14        |
|    Wii Sports Resort     |   Wii    |      2009       |    Sports    | Nintendo  |    32.77     |  15.61   |        47.64        |
| Pokemon Red/Pokemon Blue |    GB    |      1996       | Role-Playin

* Find the number NaN entries (missing data values) in each column.


In [38]:
def pretty_print_df_info_with_nans(df):
    # Show the column names, data types, & count of NaN values.
    columns_info = pd.DataFrame({
        'Column': df.columns,
        'Data Type': df.dtypes,
        'NaN Count': df.isnull().sum()
    }).reset_index(drop=True)

    # Convert 'NaN Count' column to int for better display
    columns_info['NaN Count'] = columns_info['NaN Count'].astype(int)

    table = tabulate(columns_info, headers='keys', tablefmt='pretty', showindex=False)
    print(f"Dataframe Information:\n{table}")
    
pretty_print_df_info_with_nans(video_game_rankings_df)

Dataframe Information:
+---------------------+-----------+-----------+
|       Column        | Data Type | NaN Count |
+---------------------+-----------+-----------+
|        Name         |  object   |     2     |
|      Platform       |  object   |     0     |
|   Year_of_Release   |   Int64   |    269    |
|        Genre        |  object   |     2     |
|      Publisher      |  object   |    54     |
|      NA_Sales       |  float64  |     0     |
|      EU_Sales       |  float64  |     0     |
|      JP_Sales       |  float64  |     0     |
|     Other_Sales     |  float64  |     0     |
|    Global_Sales     |  float64  |     0     |
|    Critic_Score     |  float64  |   8582    |
|    Critic_Count     |  float64  |   8582    |
|     User_Score      |  object   |   6704    |
|     User_Count      |  float64  |   9129    |
|      Developer      |  object   |   6623    |
|       Rating        |  object   |   6769    |
| NA_Sales_Percentage |  float64  |     0     |
+----------------

* Try to calculate the median user score of all the video games. You will likely run into an error because some of the user score entries are a non-numerical string that cannot be converted to a float. Find and replace this string with NaN and then calculate the median. Then, replace all NaN entries in the user score column with the median value.

In [40]:
# Convert 'User_Score' to numeric, replacing non-numeric strings with NaN
video_game_rankings_df['User_Score'] = pd.to_numeric(video_game_rankings_df['User_Score'], errors='coerce')

# Calculate the median of 'User_Score'
median_user_score = video_game_rankings_df['User_Score'].median()

# Replace NaN entries in 'User_Score' with the median value
video_game_rankings_df['User_Score'].fillna(median_user_score, inplace=True)

print(f"The median score is {median_user_score}")

The median score is 7.5
